* Modeli yalnızca bir kez eğitmek, ardından bu modeli çevrimiçi mağazamızı çalıştıran sunucuya yüklemek istiyoruz. Model, internetten indirdiğimiz bir veri seti üzerinde eğitilmiş olsa da, aslında bu veri setinde olmayan müşterilerimizin köpeklerinin önyükleme boyutunu tahmin etmek için kullanmak istiyoruz!

In [1]:
import pandas

In [2]:
data = pandas.read_csv('doggy-boot-harness.csv')
data.head()

,boot_size,harness_size,sex,age_years
0,39,58,male,12.0
1,38,58,male,9.6
2,37,52,female,8.6
3,39,58,male,10.2
4,38,57,male,7.8


#### Model oluşturma ve eğitme

In [3]:
import statsmodels.formula.api as smf

In [4]:
model = smf.ols(formula="boot_size ~ harness_size", data=data).fit()
# Bağımlı değişkenin (boot_size) bağımsız değişken (harness_size) ile olan ilişkisini ifade ediyoruz.

print("Model eğitildi!")

Model eğitildi!


#### Modeli kaydedin ve yükleyin
* Modelimiz kullanıma hazır ama henüz ihtiyacımız yok. Diske kaydedelim.

In [5]:
import joblib
# Bu modül, nesneleri diskte kaydetmek ve yüklemek için kullanılan bir işleve sahiptir.

In [6]:
model_filename = './avalanche_dog_boot_model.pkl'
joblib.dump(model, model_filename)
print("Model kaydedildi!")

Model kaydedildi!


In [7]:
model_loaded = joblib.load(model_filename)

print("Aşağıdaki parametrelere sahip bir model yükledik:")
print(model_loaded.params)

Aşağıdaki parametrelere sahip bir model yükledik:
Intercept       5.719110
harness_size    0.585925
dtype: float64


#### Bir araya getir
* Web sitemizde, müşterimizin köpeğinin koşum takımını alıp daha önce eğittiğimiz modeli kullanarak köpeğinin çizme boyutunu hesaplamak isteyeceğiz.
* Modeli diskten yükleyen ve ardından bunu müşterimizin köpeğinin çizme boyutunu tahmin etmek için kullanan bir işlev yapmak için her şeyi bir araya getirelim.

In [10]:
# Modelimizi yükleyen ve kullanan bir fonksiyon yazalım
def load_model_and_predict(harness_size):
    '''
    Bu işlev önceden eğitilmiş bir model yükler. 
    O köpeğe uyacak botların boyutunu tahmin etmek için 
    modeli müşterinin köpeğinin koşum takımı boyutuyla kullanır.
    
    harness_size: Köpek koşum takımı boyutu, cm cinsinden
    '''
    
    # Modeli dosyadan yükleyin ve bununla ilgili temel bilgileri yazdırın
    loaded_model = joblib.load(model_filename)
    
    print("Aşağıdaki parametrelere sahip bir model yükledik:")
    print(model_loaded.params)
    
    # Model için verileri hazırlayın
    inputs = {"harness_size":[harness_size]}
    
    # Tahmin yapmak için modeli kullanın
    predicted_boot_size = loaded_model.predict(inputs)[0]
    
    return predicted_boot_size

In [11]:
# Modelimizi kullanarak alıştırma yapın
predicted_boot_size = load_model_and_predict(45)

print("Öngörülen köpek çizme boyutu:", predicted_boot_size)

Aşağıdaki parametrelere sahip bir model yükledik:
Intercept       5.719110
harness_size    0.585925
dtype: float64
Öngörülen köpek çizme boyutu: 32.08575356590478


* Son adımımız, yanlış beden köpek botları alıyor olabilecekleri konusunda insanları uyarmak için bunu kullanmaktır.
* Örnek olarak, koşum boyutunu, seçilen botların boyutunu kabul eden ve müşteri için bir mesaj döndüren bir işlev yapacağız.

In [15]:
def check_size_of_boots(selected_harness_size, selected_boot_size):
    '''
    Müşterinin makul boyutta bir çift köpek botu seçip seçmediğini hesaplar. 
    Bu, köpeğin gerçek çizme boyutunu koşum boyutundan tahmin ederek çalışır.

    Bu, müşteriye ödemesini tamamlamadan önce gösterilmesi gereken bir mesaj döndürür.

    selected_harness_size: Müşterinin satın almak istediği koşum boyutu
    selected_boot_size: Müşterinin satın almak istediği köpek botlarının boyutu
    '''
    
    # Müşterinin köpeğinin çizme boyutunu tahmin edin
    estimated_boot_size = load_model_and_predict(selected_harness_size)
    
    # Parçalı ölçü satmadığımız için en yakın tam sayıya yuvarlayın
    estimated_boot_size = int(round(estimated_boot_size))
    
    # Seçilen önyükleme boyutunun uygun olup olmadığını kontrol edin
    if selected_boot_size == estimated_boot_size:
        # Seçilen çizmeler büyük olasılıkla tamamdır
        return f"Mükemmel seçim! Bu botların çığ köpeğinize çok yakışacağını düşünüyoruz."
    
    if selected_boot_size < estimated_boot_size:
        # Seçilen botlar çok küçük olabilir
        return f"Seçtiğiniz çizmeler sizinki kadar büyük bir köpek için ÇOK KÜÇÜK olabilir. {estimated_boot_size} boyutunda bir köpek botu öneriyoruz."
    
    if selected_boot_size > estimated_boot_size:
        # Seçilen botlar çok büyük olabilir
        return f"Seçtiğiniz çizmeler sizinki kadar küçük bir köpek için ÇOK BÜYÜK olabilir. {estimated_boot_size} boyutunda bir köpek botu öneriyoruz."

In [16]:
# Yeni uyarı sistemimizi kullanarak pratik yapın
check_size_of_boots(selected_harness_size=55, selected_boot_size=39)

Aşağıdaki parametrelere sahip bir model yükledik:
Intercept       5.719110
harness_size    0.585925
dtype: float64


'Seçtiğiniz çizmeler sizinki kadar küçük bir köpek için ÇOK BÜYÜK olabilir. 38 boyutunda bir köpek botu öneriyoruz.'